In [ ]:
# config=get_config('/home/python/higashi/cellcycle/1M/data/config_cellcycle.JSON')

In [31]:
####################################################################
#####################################################################
####################################################################
#######################################################################
#######对于higahsi源码，更改Higashi_analysis.py中compartment函数的定义，改为保留多个pc   pca = PCA(n_components=3)
#######多个pc无法使用原本的代码计算zscore和分位数变换，暂时只计算原始pc值
#######从label_info.pickl文件中获取细胞类型，从higahsi估计的接触矩阵按细胞类型求和得到伪bulk 并计算pc值
#######higahi计算compartment公式中的V（1）为按染色体对整个数据集求和得到的bulk数据经pca得到的投影矩阵的第一列，源代码只保留了一个pc用于后续按cell计算，
#######所有cell计算pc使用的是同一个投影矩阵，之前的sqrt_norm和oe对每个cell单独计算

###由原始的稀疏矩阵聚合后计算的pc可视化结果更接近dchic结果

from Higashi_backend.utils import *
from Higashi_analysis.Higashi_analysis import *
import h5py
from sklearn.preprocessing import MinMaxScaler, quantile_transform
import os

os.environ["OMP_NUM_THREADS"] = "10"
import pandas as pd
import argparse






def rankmatch(from_mtx, to_mtx):
	temp = np.sort(to_mtx.reshape((-1)))
	temp2 = from_mtx.reshape((-1))
	order = np.argsort(temp2)
	temp2[order] = temp
	return temp2.reshape((len(from_mtx), -1))


def create_mask(k=30, chrom="chr1", origin_sparse=None):
	final = np.array(np.sum(origin_sparse, axis=0).todense())
	size = origin_sparse[0].shape[-1]
	a = np.zeros((size, size))
	if k > 0:
		for i in range(min(k, len(a))):
			for j in range(len(a) - i):
				a[j, j + i] = 1
				a[j + i, j] = 1
		a = np.ones_like((a)) - a
	
	gap = np.sum(final, axis=-1, keepdims=False) == 0
	if cytoband_path is not None:
		gap_tab = pd.read_table(cytoband_path, sep="\t", header=None)
		gap_tab.columns = ['chrom', 'start', 'end', 'name', 'type']
		
		name = np.array(gap_tab['name'])
		# print (name)
		pqarm = np.array([str(s)[0] for s in name])
		gap_tab['pq_arm'] = pqarm
		gap_tab['length'] = gap_tab['end'] - gap_tab['start']
		summarize = gap_tab.groupby(['chrom', 'pq_arm']).sum().reset_index()
		# print (summarize)
		
		if np.sum(summarize['pq_arm'] == 'p') > 0:
			split_point = \
			np.ceil(np.array(summarize[(summarize['chrom'] == chrom) & (summarize['pq_arm'] == 'p')]['length']) / res)[0]
		else:
			split_point = -1
		
		gap_list = gap_tab[(gap_tab["chrom"] == chrom) & (gap_tab["type"] == "acen")]
		start = np.floor((np.array(gap_list['start'])) / res).astype('int')
		end = np.ceil((np.array(gap_list['end'])) / res).astype('int')
		
		for s, e in zip(start, end):
			a[s:e, :] = 1
			a[:, s:e] = 1
	else:
		split_point = -1
	a[gap, :] = 1
	a[:, gap] = 1
	
	return a, int(split_point)


def process_one_chrom(chrom):
	# Get the raw sparse mtx list
	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)
	size = origin_sparse[0].shape[0]
	# find centromere & gaps...
	mask, split_point = create_mask((int(1e5)), chrom, origin_sparse)

	bulk1 = np.array(np.sum(origin_sparse, axis=0).todense())
	# print(bulk1.shape)
	mask = (np.ones_like(bulk1) - mask)
	bulk1 *= mask

	if "bulk_path" in config:
		import cooler
		c = cooler.Cooler("%s::resolutions/%d" % (config['bulk_path'], config['resolution']))
		bulk2 = np.array(c.matrix(sparse=False, balance=False).fetch(chrom)).astype('float')
		bulk2 *= mask
	else:
		bulk2 = None

	use_rows_all = []

	if split_point >= 20 * 1000000 / res:
		slice_start_list, slice_end_list = [0, split_point], [split_point, len(bulk1)]
	else:
		slice_start_list, slice_end_list = [0], [len(bulk1)]

	bulk_compartment_all = []
	real_bulk_compartment_all = []

	bulk_model_list = []
	bulk_reverse_list = []
	bulk_slice_list = []
	use_rows_list = []
	
	temp_compartment_list_zscore = []
	temp_compartment_list_quantile = []

	for slice_start, slice_end in zip(slice_start_list, slice_end_list):
		
		bulk1_slice = bulk1[slice_start:slice_end, :]
		bulk1_slice = bulk1_slice[:, slice_start:slice_end]
		use_rows = np.where(np.sum(bulk1_slice > 0, axis=-1) > 0.01 * len(bulk1_slice))[0]
		if len(use_rows) <= 1:
			print("no reliable bins in slice:", slice_start, slice_end)
			continue
		use_rows_all.append(np.arange(slice_start, slice_end)[use_rows])
		use_rows_list.append(use_rows)
		bulk1_slice = bulk1_slice[use_rows, :]
		bulk1_slice = bulk1_slice[:, use_rows]
		
		bulk_slice_list.append(bulk1_slice)
		bulk_expect = []
		for k in range(len(bulk1_slice)):
			diag = np.diag(bulk1_slice, k)
			bulk_expect.append(np.mean(diag))
		
		
		bulk_compartment, model = test_compartment(bulk1_slice, return_PCA=True)
		
		if bulk2 is not None:
			bulk2_slice = bulk2[slice_start:slice_end, :]
			bulk2_slice = bulk2_slice[:, slice_start:slice_end]
			bulk2_slice = bulk2_slice[use_rows, :]
			bulk2_slice = bulk2_slice[:, use_rows]
			real_bulk_compartment, model = test_compartment(bulk2_slice, return_PCA=True)
		else:
			real_bulk_compartment = None
			
		reverse_flag = False
		# if args.calib:
		# 	calib = np.load(os.path.join(temp_dir, "calib.npy"), allow_pickle=True).item()[chrom].reshape((-1, 1))[slice_start:slice_end][
		# 		use_rows]
		# 	print ("average cpg", np.nanmean(calib[bulk_compartment > np.quantile(bulk_compartment,0.9)]), np.nanmean(calib[bulk_compartment < np.quantile(bulk_compartment,0.1)]))
		# 	if np.nanmean(calib[bulk_compartment > np.quantile(bulk_compartment,0.9)]) < np.nanmean(calib[bulk_compartment < np.quantile(bulk_compartment,0.1)]):
		# 		reverse_flag = True
		# 	if reverse_flag:
		# 		bulk_compartment *= -1
			
		# 	if real_bulk_compartment is not None:
		# 		reverse_flag = False
		# 		if np.nanmean(calib[real_bulk_compartment > np.quantile(real_bulk_compartment, 0.9)]) < np.nanmean(
		# 			calib[real_bulk_compartment < np.quantile(real_bulk_compartment, 0.1)]):
		# 			real_bulk_compartment *= -1
		# 			reverse_flag = True
		bulk_compartment_all.append(bulk_compartment)
		bulk_reverse_list.append(reverse_flag)
		bulk_model_list.append(model)
		real_bulk_compartment_all.append(real_bulk_compartment)
		


		
	# if args.neighbor:
	# 	impute_f = h5py.File(os.path.join(temp_dir, "%s_%s_nbr_%d_impute.hdf5" % (chrom, embedding_name, neighbor_num)),
	# 	               "r")
	# else:
	# 	impute_f =  h5py.File(os.path.join(temp_dir, "%s_%s_nbr_0_impute.hdf5" % (chrom, embedding_name)),
	# 	               "r")
	impute_f =  h5py.File(os.path.join(temp_dir, "%s_%s_nbr_0_impute.hdf5" % (chrom, embedding_name)),
						"r")
	temp_compartment_list_all = [[] for i in range(len(use_rows_list))]
	pre_bulk_list_all=[[] for i in range(len(use_rows_list))]
	
	coordinates = impute_f['coordinates']
	xs, ys = coordinates[:, 0], coordinates[:, 1]
	cell_list = trange(len(list(impute_f.keys())) - 1)
	temp = np.zeros((size, size))

	for i in cell_list:
		temp *= 0.0
		proba = np.array(impute_f["cell_%d" % i])
		temp[xs.astype('int'), ys.astype('int')] += proba
		temp = temp + temp.T
		temp *= mask
		
		for j in range(len(use_rows_list)):
			slice_start, slice_end = slice_start_list[j], slice_end_list[j]
			temp_slice = temp[slice_start:slice_end, :]
			temp_slice = temp_slice[:, slice_start:slice_end]
			temp_select = temp_slice[use_rows_list[j], :]
			temp_select = temp_select[:, use_rows_list[j]]
			# temp_select = rankmatch(temp_select, bulk_slice_list[j])
			temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
			if bulk_reverse_list[j]:
				temp_compartment = -1 * temp_compartment
			# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
			temp_compartment_list_all[j].append(temp_compartment)
			
	for j in range(len(use_rows_list)):
		temp_compartment_list_all[j] = np.stack(temp_compartment_list_all[j], axis=0)
		# temp_compartment_list_quantile.append(quantile_transform(temp_compartment_list_all[j], output_distribution='uniform',
		#                                            n_quantiles=int(temp_compartment_list_all[j].shape[-1] * 1.0), axis=1))
		
		# temp_compartment_list_zscore.append(zscore(temp_compartment_list_all[j], axis=1))

	# print(bulk_compartment.shape, temp_compartment_list.shape)


	real_bulk_compartment = np.concatenate(real_bulk_compartment_all, axis=0) if bulk2 is not None else None
	bulk_compartment = np.concatenate(bulk_compartment_all, axis=0)
	temp_compartment_list = np.concatenate(temp_compartment_list_all, axis=-1)
	# temp_compartment_list_zscore = np.concatenate(temp_compartment_list_zscore, axis=-1)
	# temp_compartment_list_quantile = np.concatenate(temp_compartment_list_quantile, axis=-1)
	use_rows = np.concatenate(use_rows_all, axis=0)
	print (chrom, "finished")
	# print("temp_compartment_list shape:",temp_compartment_list.shape)

	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	# sum_by_label=[]
	a={}
	b={}
	for j in list(set(cell_type)):
		
		indices = [index for index, value in enumerate(cell_type) if value == j]
		a[j]=indices
		b[j] = np.zeros_like(np.array(impute_f["cell_0"]))
		for i in a[j]:
				proba = np.array(impute_f["cell_%d" % i])
				b[j] +=proba
		temp *= 0.0
		temp[xs.astype('int'), ys.astype('int')] += b[j]
		temp = temp + temp.T
		temp *= mask
		
		for j in range(len(use_rows_list)):
			slice_start, slice_end = slice_start_list[j], slice_end_list[j]
			temp_slice = temp[slice_start:slice_end, :]
			temp_slice = temp_slice[:, slice_start:slice_end]
			temp_select = temp_slice[use_rows_list[j], :]
			temp_select = temp_select[:, use_rows_list[j]]
			# temp_select = rankmatch(temp_select, bulk_slice_list[j])
			temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
			if bulk_reverse_list[j]:
				temp_compartment = -1 * temp_compartment
			# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
			pre_bulk_list_all[j].append(temp_compartment)		
	for j in range(len(use_rows_list)):
		pre_bulk_list_all[j] = np.stack(pre_bulk_list_all[j], axis=0)
	pre_bulk_list = np.concatenate(pre_bulk_list_all, axis=-1)

	return real_bulk_compartment, bulk_compartment, temp_compartment_list, chrom, use_rows, size,pre_bulk_list

	
			

def process_calib_file(file_path):
	tab = pd.read_table(file_path , sep="\t", header=None)
	tab.columns = ['chrom', 'bin', 'value']
	# print(tab)
	chrom_start_end = np.load(os.path.join(temp_dir, "chrom_start_end.npy"))
	tab['chrom'] = np.array(tab['chrom']).astype('str')
	calib_result = {}
	for i, chrom in enumerate(chrom_list):
		temp = tab[tab['chrom'] == chrom]
		size = chrom_start_end[i, 1] - chrom_start_end[i, 0]
		vec = np.zeros(size)
		indice = (np.array(temp['bin'] / res)).astype('int')
		v = np.array(temp['value'])
		v[v == -1] = np.nan
		vec[indice] = v
		calib_result[chrom] = vec
	np.save(os.path.join(temp_dir, "calib.npy"), calib_result, allow_pickle=True)


def start_call_compartment():
	p_list = []
	pool = ProcessPoolExecutor(max_workers=10)
	output = 'cycle1M'
	if ".hdf5" not in output:
		output += ".hdf5"
	with h5py.File(os.path.join(temp_dir, output), "w") as output_f:
		result = {}
		for chrom in chrom_list:
			# real_bulk_compartment, bulk_compartment, temp_compartment_list, temp_compartment_zscore, temp_compartment_quantile, chrom, use_rows, size = process_one_chrom(chrom)
			real_bulk_compartment, bulk_compartment, temp_compartment_list, chrom, use_rows, size,pre_bulk_list = process_one_chrom(chrom)
		# 	p_list.append(pool.submit(process_one_chrom, chrom))
		# 
		# 
		# for p in as_completed(p_list):
		# 	real_bulk_compartment, bulk_compartment, temp_compartment_list, temp_compartment_zscore, temp_compartment_quantile, chrom, use_rows, size = p.result()
			result[chrom] = [real_bulk_compartment, bulk_compartment, temp_compartment_list, use_rows, size,pre_bulk_list]
			
		bin_chrom_list = []
		bin_start_list = []
		bin_end_list = []
		bulk_cp_all = []
		real_bulk_cp_all = []
		sc_cp_all = []
		sc_cp_raw = []
		sc_cp_zscore = []
		sc_cp_bulk = []
		grp = output_f.create_group('compartment')
		bin = grp.create_group('bin')
		
		for chrom in chrom_list:
			real_bulk_compartment, bulk_compartment, temp_compartment_list, use_rows, size,pre_bulk_list = result[chrom]
			# print (use_rows)
			# print(chrom)
			length = size
			bin_chrom_list += [chrom] * len(use_rows)
			bin_start_list.append((np.arange(length) * res).astype('int')[use_rows])
			bin_end_list.append(((np.arange(length) + 1) * res).astype('int')[use_rows])
			bulk_cp_all.append(bulk_compartment)
			real_bulk_cp_all.append(real_bulk_compartment)
			# sc_cp_all.append(temp_compartment_quantile)
			
			# print("temp_compartment_list shap to append:",temp_compartment_list.shape)
			sc_cp_raw.append(temp_compartment_list)
			# print("sc_cp_raw shape",(np.array(sc_cp_raw)).shape)
			sc_cp_bulk.append(pre_bulk_list)
			
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		
		bulk_cp_all = np.concatenate(bulk_cp_all, axis=0)
		grp.create_dataset("bulk", data=bulk_cp_all)
		
		if real_bulk_compartment is not None:
			real_bulk_cp_all = np.concatenate(real_bulk_cp_all, axis=0)
			grp.create_dataset("real_bulk", data=real_bulk_cp_all)
		
		# sc_cp_all = np.concatenate(sc_cp_all, axis=-1)
		sc_cp_raw = np.concatenate(sc_cp_raw, axis=-2)
		# print('sc_cp_raw:',(np.array(sc_cp_raw)).shape)
		# sc_cp_zscore = np.concatenate(sc_cp_zscore, axis=-1)
		sc_cp_bulk=np.concatenate(sc_cp_bulk, axis=-2)
		
		# for cell in range(len(sc_cp_all)):
		# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_all[cell])
		
		grp = output_f.create_group('compartment_raw')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		for cell in range(len(sc_cp_raw)):
			grp.create_dataset("cell_%d" % cell, data=sc_cp_raw[cell])
		
		# grp = output_f.create_group('compartment_zscore')
		# bin = grp.create_group('bin')
		# bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		#                    dtype=h5py.special_dtype(vlen=str))
		# bin.create_dataset('start', data=np.concatenate(bin_start_list))
		# bin.create_dataset('end', data=np.concatenate(bin_end_list))
		# for cell in range(len(sc_cp_all)):
		# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_zscore[cell])
		grp = output_f.create_group('compartment_bulk')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
		cell_type=cell_type_info['cell_type']
		i=0
		for j in list(set(cell_type)):
			grp.create_dataset(j, data=sc_cp_bulk[i])
			i+=1

			

	output_f.close()
	pool.shutdown(wait=True)

def test_compartment(matrix, return_PCA=False, model=None, expected = None):
	contact = matrix
	# np.fill_diagonal(contact, np.max(contact))
	# contact = KRnormalize(matrix)
	# contact[np.isnan(contact)] = 0.0
	contact = sqrt_norm(matrix)
	contact = oe(contact, expected)
	np.fill_diagonal(contact, 1)
	with warnings.catch_warnings():
		warnings.filterwarnings(
			"ignore", category=PearsonRConstantInputWarning
		)
		contact = pearson(contact)
	np.fill_diagonal(contact, 1)
	contact[np.isnan(contact)] = 0.0
	if model is not None:
		y = model.transform(contact)
	else:
		pca = PCA(n_components=3)
		y = pca.fit_transform(contact)
	if return_PCA:
		return y, pca
	else:
		return y




In [32]:
# args = parse_args()
# print (args)
config = get_config('/home/python/higashi/cellcycle/1M/data/config_cellcycle.JSON')
# config=get_config('/home/python/higashi/dataset2/cortex_250k_tmp/config_cellcycle.json')
res = config['resolution']
data_dir = config['data_dir']
temp_dir = config['temp_dir']
raw_dir = os.path.join(temp_dir, "raw")
if 'cytoband_path' in config:
	cytoband_path = config['cytoband_path']
else:
	cytoband_path = None
neighbor_num = config['neighbor_num']
embedding_name = config['embedding_name']

chrom_list = config['impute_list']
chrom_list = np.array(chrom_list)


# if args.calib:
# 	process_calib_file(args.calib_file)
start_call_compartment()















































































































































































































































 81%|████████  | 945/1171 [00:30<00:07, 31.26it/s]


KeyboardInterrupt: 

In [42]:
# chrom='chr1'
# impute_f =  h5py.File(os.path.join(temp_dir, "%s_%s_nbr_0_impute.hdf5" % (chrom, embedding_name)),
#                         "r")
# coordinates = impute_f['coordinates']
# xs, ys = coordinates[:, 0], coordinates[:, 1]
# cell_list = trange(len(list(impute_f.keys())) - 1)

# proba = np.array(impute_f["cell_%d" % 0])
# type(proba)

# # cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
# # cell_type=cell_type_info['cell_type']
# # # sum_by_label=[]
# # a={}
# # b={}
# # for j in list(set(cell_type)):
    
# #     indices = [index for index, value in enumerate(cell_type) if value == j]
# #     a[j]=indices
# #     b[j] = np.zeros_like(np.array(impute_f["cell_0"]))
# #     for i in a[j]:
# #             proba = np.array(impute_f["cell_%d" % i])
# #             b[j] +=proba 


  0%|          | 0/1171 [00:15<?, ?it/s]


numpy.ndarray

In [40]:
# b['1CDX1']

array([2.9971005e-04, 1.1129287e-03, 8.7686487e-05, ..., 1.9357675e+02,
       8.0494614e+01, 2.3299082e+01], dtype=float32)

In [29]:
# a={}
# for j in list(set(cell_type)):

#     indices = [index for index, value in enumerate(cell_type) if value == j]
    
#     a[j]=indices
#     print(j,a[j])

1CDX3 [583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781

In [32]:
# for c in a[j]:
#     print(c)

845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075


In [31]:
# type(a[j])

list

In [18]:
# a=list(set(cell_type))

In [20]:
# len(a)

4

In [51]:

# cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
# cell_type=cell_type_info['cell_type']
# origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)
# size = origin_sparse[0].shape[0]
# # find centromere & gaps...
# mask, split_point = create_mask((int(1e5)), chrom, origin_sparse)

# bulk1 = np.array(np.sum(origin_sparse, axis=0).todense())
# print(bulk1.shape)
# mask = (np.ones_like(bulk1) - mask)
# bulk1 *= mask

# if "bulk_path" in config:
#     import cooler
#     c = cooler.Cooler("%s::resolutions/%d" % (config['bulk_path'], config['resolution']))
#     bulk2 = np.array(c.matrix(sparse=False, balance=False).fetch(chrom)).astype('float')
#     bulk2 *= mask
# else:
#     bulk2 = None

# use_rows_all = []

# if split_point >= 20 * 1000000 / res:
#     slice_start_list, slice_end_list = [0, split_point], [split_point, len(bulk1)]
# else:
#     slice_start_list, slice_end_list = [0], [len(bulk1)]

# bulk_compartment_all = []
# real_bulk_compartment_all = []

# bulk_model_list = []
# bulk_reverse_list = []
# bulk_slice_list = []
# use_rows_list = []

# temp_compartment_list_zscore = []
# temp_compartment_list_quantile = []

# for slice_start, slice_end in zip(slice_start_list, slice_end_list):
    
#     bulk1_slice = bulk1[slice_start:slice_end, :]
#     bulk1_slice = bulk1_slice[:, slice_start:slice_end]
#     use_rows = np.where(np.sum(bulk1_slice > 0, axis=-1) > 0.01 * len(bulk1_slice))[0]
#     if len(use_rows) <= 1:
#         print("no reliable bins in slice:", slice_start, slice_end)
#         continue
#     use_rows_all.append(np.arange(slice_start, slice_end)[use_rows])
#     use_rows_list.append(use_rows)
#     bulk1_slice = bulk1_slice[use_rows, :]
#     bulk1_slice = bulk1_slice[:, use_rows]
    
#     bulk_slice_list.append(bulk1_slice)
#     bulk_expect = []
#     for k in range(len(bulk1_slice)):
#         diag = np.diag(bulk1_slice, k)
#         bulk_expect.append(np.mean(diag))
    
    
#     bulk_compartment, model = test_compartment(bulk1_slice, return_PCA=True)
    
#     if bulk2 is not None:
#         bulk2_slice = bulk2[slice_start:slice_end, :]
#         bulk2_slice = bulk2_slice[:, slice_start:slice_end]
#         bulk2_slice = bulk2_slice[use_rows, :]
#         bulk2_slice = bulk2_slice[:, use_rows]
#         real_bulk_compartment, model = test_compartment(bulk2_slice, return_PCA=True)
#     else:
#         real_bulk_compartment = None
        
#     reverse_flag = False
#     # if args.calib:
#     # 	calib = np.load(os.path.join(temp_dir, "calib.npy"), allow_pickle=True).item()[chrom].reshape((-1, 1))[slice_start:slice_end][
#     # 		use_rows]
#     # 	print ("average cpg", np.nanmean(calib[bulk_compartment > np.quantile(bulk_compartment,0.9)]), np.nanmean(calib[bulk_compartment < np.quantile(bulk_compartment,0.1)]))
#     # 	if np.nanmean(calib[bulk_compartment > np.quantile(bulk_compartment,0.9)]) < np.nanmean(calib[bulk_compartment < np.quantile(bulk_compartment,0.1)]):
#     # 		reverse_flag = True
#     # 	if reverse_flag:
#     # 		bulk_compartment *= -1
        
#     # 	if real_bulk_compartment is not None:
#     # 		reverse_flag = False
#     # 		if np.nanmean(calib[real_bulk_compartment > np.quantile(real_bulk_compartment, 0.9)]) < np.nanmean(
#     # 			calib[real_bulk_compartment < np.quantile(real_bulk_compartment, 0.1)]):
#     # 			real_bulk_compartment *= -1
#     # 			reverse_flag = True
#     bulk_compartment_all.append(bulk_compartment)
#     bulk_reverse_list.append(reverse_flag)
#     bulk_model_list.append(model)
#     real_bulk_compartment_all.append(real_bulk_compartment)
    


    
# # if args.neighbor:
# # 	impute_f = h5py.File(os.path.join(temp_dir, "%s_%s_nbr_%d_impute.hdf5" % (chrom, embedding_name, neighbor_num)),
# # 	               "r")
# # else:
# # 	impute_f =  h5py.File(os.path.join(temp_dir, "%s_%s_nbr_0_impute.hdf5" % (chrom, embedding_name)),
# # 	               "r")
# impute_f =  h5py.File(os.path.join(temp_dir, "%s_%s_nbr_0_impute.hdf5" % (chrom, embedding_name)),
#                     "r")
# temp_compartment_list_all = [[] for i in range(len(use_rows_list))]
# pre_bulk_list_all=[[] for i in range(len(use_rows_list))]

# coordinates = impute_f['coordinates']
# xs, ys = coordinates[:, 0], coordinates[:, 1]
# cell_list = trange(len(list(impute_f.keys())) - 1)
# temp = np.zeros((size, size))

# for i in cell_list:
#     temp *= 0.0
#     proba = np.array(impute_f["cell_%d" % i])
#     temp[xs.astype('int'), ys.astype('int')] += proba
#     temp = temp + temp.T
#     temp *= mask
    
#     for j in range(len(use_rows_list)):
#         slice_start, slice_end = slice_start_list[j], slice_end_list[j]
#         temp_slice = temp[slice_start:slice_end, :]
#         temp_slice = temp_slice[:, slice_start:slice_end]
#         temp_select = temp_slice[use_rows_list[j], :]
#         temp_select = temp_select[:, use_rows_list[j]]
#         # temp_select = rankmatch(temp_select, bulk_slice_list[j])
#         temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
#         if bulk_reverse_list[j]:
#             temp_compartment = -1 * temp_compartment
#         # temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
#         temp_compartment_list_all[j].append(temp_compartment)
        
# for j in range(len(use_rows_list)):
#     temp_compartment_list_all[j] = np.stack(temp_compartment_list_all[j], axis=0)
#     # temp_compartment_list_quantile.append(quantile_transform(temp_compartment_list_all[j], output_distribution='uniform',
#     #                                            n_quantiles=int(temp_compartment_list_all[j].shape[-1] * 1.0), axis=1))
    
#     # temp_compartment_list_zscore.append(zscore(temp_compartment_list_all[j], axis=1))

# # print(bulk_compartment.shape, temp_compartment_list.shape)


# real_bulk_compartment = np.concatenate(real_bulk_compartment_all, axis=0) if bulk2 is not None else None
# bulk_compartment = np.concatenate(bulk_compartment_all, axis=0)
# temp_compartment_list = np.concatenate(temp_compartment_list_all, axis=-1)
# # temp_compartment_list_zscore = np.concatenate(temp_compartment_list_zscore, axis=-1)
# # temp_compartment_list_quantile = np.concatenate(temp_compartment_list_quantile, axis=-1)
# use_rows = np.concatenate(use_rows_all, axis=0)
# print (chrom, "finished")
# print("temp_compartment_list shape:",temp_compartment_list.shape)

# cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
# cell_type=cell_type_info['cell_type']
# # sum_by_label=[]
# a={}
# b={}
# for j in list(set(cell_type)):
    
#     indices = [index for index, value in enumerate(cell_type) if value == j]
#     a[j]=indices
#     b[j] = np.zeros_like(np.array(impute_f["cell_0"]))
#     for i in a[j]:
#             proba = np.array(impute_f["cell_%d" % i])
#             b[j] +=proba
#     temp *= 0.0
#     temp[xs.astype('int'), ys.astype('int')] += b[j]
#     temp = temp + temp.T
#     temp *= mask
    
#     for j in range(len(use_rows_list)):
#         slice_start, slice_end = slice_start_list[j], slice_end_list[j]
#         temp_slice = temp[slice_start:slice_end, :]
#         temp_slice = temp_slice[:, slice_start:slice_end]
#         temp_select = temp_slice[use_rows_list[j], :]
#         temp_select = temp_select[:, use_rows_list[j]]
#         # temp_select = rankmatch(temp_select, bulk_slice_list[j])
#         temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
#         if bulk_reverse_list[j]:
#             temp_compartment = -1 * temp_compartment
#         # temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
#         pre_bulk_list_all[j].append(temp_compartment)		
# for j in range(len(use_rows_list)):
#     pre_bulk_list_all[j] = np.stack(pre_bulk_list_all[j], axis=0)
# pre_bulk_list = np.concatenate(pre_bulk_list_all, axis=-1)

(198, 198)


100%|██████████| 1171/1171 [00:36<00:00, 32.15it/s]


chr1 finished
temp_compartment_list shape: (1171, 194, 3)


In [47]:
# j

0

In [49]:
# pre_bulk_list[3].shape

(194, 3)

In [50]:
# for j in list(set(cell_type)):
#     print(j)

1CDX3
1CDX1
1CDX2
1CDX4


In [125]:
############################################################
#使用原始的稀疏矩阵计算compartment
##########################################################
#############################################################
#############################################################
##################################################################
##################################################################
from Higashi_backend.utils import *
from Higashi_analysis.Higashi_analysis import *
import h5py
from sklearn.preprocessing import MinMaxScaler, quantile_transform
import os

os.environ["OMP_NUM_THREADS"] = "10"
import pandas as pd
import argparse






def rankmatch(from_mtx, to_mtx):
	temp = np.sort(to_mtx.reshape((-1)))
	temp2 = from_mtx.reshape((-1))
	order = np.argsort(temp2)
	temp2[order] = temp
	return temp2.reshape((len(from_mtx), -1))


def create_mask(k=30, chrom="chr1", origin_sparse=None):
	final = np.array(np.sum(origin_sparse, axis=0).todense())
	size = origin_sparse[0].shape[-1]
	a = np.zeros((size, size))
	if k > 0:
		for i in range(min(k, len(a))):
			for j in range(len(a) - i):
				a[j, j + i] = 1
				a[j + i, j] = 1
		a = np.ones_like((a)) - a
	
	gap = np.sum(final, axis=-1, keepdims=False) == 0
	if cytoband_path is not None:
		gap_tab = pd.read_table(cytoband_path, sep="\t", header=None)
		gap_tab.columns = ['chrom', 'start', 'end', 'name', 'type']
		
		name = np.array(gap_tab['name'])
		# print (name)
		pqarm = np.array([str(s)[0] for s in name])
		gap_tab['pq_arm'] = pqarm
		gap_tab['length'] = gap_tab['end'] - gap_tab['start']
		summarize = gap_tab.groupby(['chrom', 'pq_arm']).sum().reset_index()
		# print (summarize)
		
		if np.sum(summarize['pq_arm'] == 'p') > 0:
			split_point = \
			np.ceil(np.array(summarize[(summarize['chrom'] == chrom) & (summarize['pq_arm'] == 'p')]['length']) / res)[0]
		else:
			split_point = -1
		
		gap_list = gap_tab[(gap_tab["chrom"] == chrom) & (gap_tab["type"] == "acen")]
		start = np.floor((np.array(gap_list['start'])) / res).astype('int')
		end = np.ceil((np.array(gap_list['end'])) / res).astype('int')
		
		for s, e in zip(start, end):
			a[s:e, :] = 1
			a[:, s:e] = 1
	else:
		split_point = -1
	a[gap, :] = 1
	a[:, gap] = 1
	
	return a, int(split_point)


def process_one_chrom(chrom):
	# Get the raw sparse mtx list
	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)
	size = origin_sparse[0].shape[0]
	# find centromere & gaps...
	mask, split_point = create_mask((int(1e5)), chrom, origin_sparse)

	bulk1 = np.array(np.sum(origin_sparse, axis=0).todense())
	print(bulk1.shape)
	mask = (np.ones_like(bulk1) - mask)
	bulk1 *= mask

	if "bulk_path" in config:
		import cooler
		c = cooler.Cooler("%s::resolutions/%d" % (config['bulk_path'], config['resolution']))
		bulk2 = np.array(c.matrix(sparse=False, balance=False).fetch(chrom)).astype('float')
		bulk2 *= mask
	else:
		bulk2 = None

	use_rows_all = []

	if split_point >= 20 * 1000000 / res:
		slice_start_list, slice_end_list = [0, split_point], [split_point, len(bulk1)]
	else:
		slice_start_list, slice_end_list = [0], [len(bulk1)]

	bulk_compartment_all = []
	real_bulk_compartment_all = []

	bulk_model_list = []
	bulk_reverse_list = []
	bulk_slice_list = []
	use_rows_list = []
	
	temp_compartment_list_zscore = []
	temp_compartment_list_quantile = []

	for slice_start, slice_end in zip(slice_start_list, slice_end_list):
		
		bulk1_slice = bulk1[slice_start:slice_end, :]
		bulk1_slice = bulk1_slice[:, slice_start:slice_end]
		use_rows = np.where(np.sum(bulk1_slice > 0, axis=-1) > 0.01 * len(bulk1_slice))[0]
		if len(use_rows) <= 1:
			print("no reliable bins in slice:", slice_start, slice_end)
			continue
		use_rows_all.append(np.arange(slice_start, slice_end)[use_rows])
		use_rows_list.append(use_rows)
		bulk1_slice = bulk1_slice[use_rows, :]
		bulk1_slice = bulk1_slice[:, use_rows]
		
		bulk_slice_list.append(bulk1_slice)
		bulk_expect = []
		for k in range(len(bulk1_slice)):
			diag = np.diag(bulk1_slice, k)
			bulk_expect.append(np.mean(diag))
		
		
		bulk_compartment, model = test_compartment(bulk1_slice, return_PCA=True)
		
		if bulk2 is not None:
			bulk2_slice = bulk2[slice_start:slice_end, :]
			bulk2_slice = bulk2_slice[:, slice_start:slice_end]
			bulk2_slice = bulk2_slice[use_rows, :]
			bulk2_slice = bulk2_slice[:, use_rows]
			real_bulk_compartment, model = test_compartment(bulk2_slice, return_PCA=True)
		else:
			real_bulk_compartment = None
			
		reverse_flag = False
		# if args.calib:
		# 	calib = np.load(os.path.join(temp_dir, "calib.npy"), allow_pickle=True).item()[chrom].reshape((-1, 1))[slice_start:slice_end][
		# 		use_rows]
		# 	print ("average cpg", np.nanmean(calib[bulk_compartment > np.quantile(bulk_compartment,0.9)]), np.nanmean(calib[bulk_compartment < np.quantile(bulk_compartment,0.1)]))
		# 	if np.nanmean(calib[bulk_compartment > np.quantile(bulk_compartment,0.9)]) < np.nanmean(calib[bulk_compartment < np.quantile(bulk_compartment,0.1)]):
		# 		reverse_flag = True
		# 	if reverse_flag:
		# 		bulk_compartment *= -1
			
		# 	if real_bulk_compartment is not None:
		# 		reverse_flag = False
		# 		if np.nanmean(calib[real_bulk_compartment > np.quantile(real_bulk_compartment, 0.9)]) < np.nanmean(
		# 			calib[real_bulk_compartment < np.quantile(real_bulk_compartment, 0.1)]):
		# 			real_bulk_compartment *= -1
		# 			reverse_flag = True
		bulk_compartment_all.append(bulk_compartment)
		bulk_reverse_list.append(reverse_flag)
		bulk_model_list.append(model)
		real_bulk_compartment_all.append(real_bulk_compartment)
		


		
	# if args.neighbor:
	# 	impute_f = h5py.File(os.path.join(temp_dir, "%s_%s_nbr_%d_impute.hdf5" % (chrom, embedding_name, neighbor_num)),
	# 	               "r")
	# else:
	# 	impute_f =  h5py.File(os.path.join(temp_dir, "%s_%s_nbr_0_impute.hdf5" % (chrom, embedding_name)),
	# 	               "r")
	# impute_f =  h5py.File(os.path.join(temp_dir, "%s_%s_nbr_0_impute.hdf5" % (chrom, embedding_name)),
	# 					"r")
	# temp_compartment_list_all = [[] for i in range(len(use_rows_list))]
	# pre_bulk_list_all=[[] for i in range(len(use_rows_list))]
	
	# coordinates = impute_f['coordinates']
	# xs, ys = coordinates[:, 0], coordinates[:, 1]
	# cell_list = trange(len(list(impute_f.keys())) - 1)
	# temp = np.zeros((size, size))

	# for i in cell_list:
	# 	temp *= 0.0
	# 	proba = np.array(impute_f["cell_%d" % i])
	# 	temp[xs.astype('int'), ys.astype('int')] += proba
	# 	temp = temp + temp.T
	# 	temp *= mask
		
	# 	for j in range(len(use_rows_list)):
	# 		slice_start, slice_end = slice_start_list[j], slice_end_list[j]
	# 		temp_slice = temp[slice_start:slice_end, :]
	# 		temp_slice = temp_slice[:, slice_start:slice_end]
	# 		temp_select = temp_slice[use_rows_list[j], :]
	# 		temp_select = temp_select[:, use_rows_list[j]]
	# 		# temp_select = rankmatch(temp_select, bulk_slice_list[j])
	# 		temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
	# 		if bulk_reverse_list[j]:
	# 			temp_compartment = -1 * temp_compartment
	# 		# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
	# 		temp_compartment_list_all[j].append(temp_compartment)
			
	# for j in range(len(use_rows_list)):
	# 	temp_compartment_list_all[j] = np.stack(temp_compartment_list_all[j], axis=0)
		# temp_compartment_list_quantile.append(quantile_transform(temp_compartment_list_all[j], output_distribution='uniform',
		#                                            n_quantiles=int(temp_compartment_list_all[j].shape[-1] * 1.0), axis=1))
		
		# temp_compartment_list_zscore.append(zscore(temp_compartment_list_all[j], axis=1))

	# print(bulk_compartment.shape, temp_compartment_list.shape)


	real_bulk_compartment = np.concatenate(real_bulk_compartment_all, axis=0) if bulk2 is not None else None
	bulk_compartment = np.concatenate(bulk_compartment_all, axis=0)
	# temp_compartment_list = np.concatenate(temp_compartment_list_all, axis=-1)
	# temp_compartment_list_zscore = np.concatenate(temp_compartment_list_zscore, axis=-1)
	# temp_compartment_list_quantile = np.concatenate(temp_compartment_list_quantile, axis=-1)
	use_rows = np.concatenate(use_rows_all, axis=0)
	# print (chrom, "finished")
	# print("temp_compartment_list shape:",temp_compartment_list.shape)

	##########从原始的接触图计算
	origin_bulk_list_all=[[] for i in range(len(use_rows_list))]
	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	# sum_by_label=[]
	a={}
	b={}
	for j in list(set(cell_type)):
		
		indices = [index for index, value in enumerate(cell_type) if value == j]
		a[j]=indices
		b[j] = np.zeros_like(np.array(origin_sparse[0]))
		for i in a[j]:
				proba = np.array(origin_sparse[i])
				b[j] +=proba
		temp = np.array(b[j].item().todense())
		# print(temp.shape)
		temp *= mask
		for j in range(len(use_rows_list)):
			slice_start, slice_end = slice_start_list[j], slice_end_list[j]
			temp_slice = temp[slice_start:slice_end, :]
			temp_slice = temp_slice[:, slice_start:slice_end]
			temp_select = temp_slice[use_rows_list[j], :]
			temp_select = temp_select[:, use_rows_list[j]]
			# temp_select = rankmatch(temp_select, bulk_slice_list[j])
			temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
			if bulk_reverse_list[j]:
				temp_compartment = -1 * temp_compartment
			# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
			origin_bulk_list_all[j].append(temp_compartment)
        
	for j in range(len(use_rows_list)):
		origin_bulk_list_all[j] = np.stack(origin_bulk_list_all[j], axis=0)
	origin_bulk_list = np.stack(origin_bulk_list_all[j], axis=-1)
		


######从估计的接触图的伪bulk计算compartment
	# cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	# cell_type=cell_type_info['cell_type']
	# # sum_by_label=[]
	# a={}
	# b={}
	# for j in list(set(cell_type)):
		
	# 	indices = [index for index, value in enumerate(cell_type) if value == j]
	# 	a[j]=indices
	# 	b[j] = np.zeros_like(np.array(impute_f["cell_0"]))
	# 	for i in a[j]:
	# 			proba = np.array(impute_f["cell_%d" % i])
	# 			b[j] +=proba
	# 	temp *= 0.0
	# 	temp[xs.astype('int'), ys.astype('int')] += b[j]
	# 	temp = temp + temp.T
	# 	temp *= mask
		
	# 	for j in range(len(use_rows_list)):
	# 		slice_start, slice_end = slice_start_list[j], slice_end_list[j]
	# 		temp_slice = temp[slice_start:slice_end, :]
	# 		temp_slice = temp_slice[:, slice_start:slice_end]
	# 		temp_select = temp_slice[use_rows_list[j], :]
	# 		temp_select = temp_select[:, use_rows_list[j]]
	# 		# temp_select = rankmatch(temp_select, bulk_slice_list[j])
	# 		temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
	# 		if bulk_reverse_list[j]:
	# 			temp_compartment = -1 * temp_compartment
	# 		# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
	# 		pre_bulk_list_all[j].append(temp_compartment)		
	# for j in range(len(use_rows_list)):
	# 	pre_bulk_list_all[j] = np.stack(pre_bulk_list_all[j], axis=0)
	# pre_bulk_list = np.concatenate(pre_bulk_list_all, axis=-1)

	return real_bulk_compartment, bulk_compartment, chrom, use_rows, size,origin_bulk_list

	
			

def process_calib_file(file_path):
	tab = pd.read_table(file_path , sep="\t", header=None)
	tab.columns = ['chrom', 'bin', 'value']
	# print(tab)
	chrom_start_end = np.load(os.path.join(temp_dir, "chrom_start_end.npy"))
	tab['chrom'] = np.array(tab['chrom']).astype('str')
	calib_result = {}
	for i, chrom in enumerate(chrom_list):
		temp = tab[tab['chrom'] == chrom]
		size = chrom_start_end[i, 1] - chrom_start_end[i, 0]
		vec = np.zeros(size)
		indice = (np.array(temp['bin'] / res)).astype('int')
		v = np.array(temp['value'])
		v[v == -1] = np.nan
		vec[indice] = v
		calib_result[chrom] = vec
	np.save(os.path.join(temp_dir, "calib.npy"), calib_result, allow_pickle=True)


def start_call_compartment():
	p_list = []
	pool = ProcessPoolExecutor(max_workers=10)
	output = 'ori_bulk_cortex250k'
	if ".hdf5" not in output:
		output += ".hdf5"
	with h5py.File(os.path.join(temp_dir, output), "w") as output_f:
		result = {}
		for chrom in chrom_list:
			# real_bulk_compartment, bulk_compartment, temp_compartment_list, temp_compartment_zscore, temp_compartment_quantile, chrom, use_rows, size = process_one_chrom(chrom)
			real_bulk_compartment, bulk_compartment, chrom, use_rows, size,origin_bulk_list = process_one_chrom(chrom)
		# 	p_list.append(pool.submit(process_one_chrom, chrom))
		# 
		# 
		# for p in as_completed(p_list):
		# 	real_bulk_compartment, bulk_compartment, temp_compartment_list, temp_compartment_zscore, temp_compartment_quantile, chrom, use_rows, size = p.result()
			result[chrom] = [real_bulk_compartment, bulk_compartment, use_rows, size,origin_bulk_list]
			
		bin_chrom_list = []
		bin_start_list = []
		bin_end_list = []
		bulk_cp_all = []
		real_bulk_cp_all = []
		sc_cp_all = []
		sc_cp_raw = []
		sc_cp_zscore = []
		sc_cp_bulk = []
		grp = output_f.create_group('compartment')
		bin = grp.create_group('bin')
		
		for chrom in chrom_list:
			real_bulk_compartment, bulk_compartment, use_rows, size,origin_bulk_list = result[chrom]
			# print (use_rows)
			print(chrom)
			length = size
			bin_chrom_list += [chrom] * len(use_rows)
			bin_start_list.append((np.arange(length) * res).astype('int')[use_rows])
			bin_end_list.append(((np.arange(length) + 1) * res).astype('int')[use_rows])
			bulk_cp_all.append(bulk_compartment)
			real_bulk_cp_all.append(real_bulk_compartment)
			# sc_cp_all.append(temp_compartment_quantile)
			
			# print("temp_compartment_list shap to append:",temp_compartment_list.shape)
			# sc_cp_raw.append(temp_compartment_list)
			# print("sc_cp_raw shape",(np.array(sc_cp_raw)).shape)
			sc_cp_bulk.append(origin_bulk_list)
			
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		# print("bulk_cp_all:",np.array(bulk_cp_all).shape)
		bulk_cp_all = np.concatenate(bulk_cp_all, axis=0)
		# print("bulk_cp_all shape:",np.array(bulk_cp_all).shape)
		grp.create_dataset("bulk", data=bulk_cp_all)
		
		if real_bulk_compartment is not None:
			real_bulk_cp_all = np.concatenate(real_bulk_cp_all, axis=0)
			grp.create_dataset("real_bulk", data=real_bulk_cp_all)
		
		# sc_cp_all = np.concatenate(sc_cp_all, axis=-1)
		# sc_cp_raw = np.concatenate(sc_cp_raw, axis=-2)
		# print('sc_cp_raw:',(np.array(sc_cp_raw)).shape)
		# sc_cp_zscore = np.concatenate(sc_cp_zscore, axis=-1)
		sc_cp_bulk=np.concatenate(sc_cp_bulk, axis=0)
		# print('sc_cp_bulk:',(np.array(sc_cp_bulk)).shape)
		
		# for cell in range(len(sc_cp_all)):
		# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_all[cell])
		
		# grp = output_f.create_group('compartment_raw')
		# bin = grp.create_group('bin')
		# bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		#                    dtype=h5py.special_dtype(vlen=str))
		# bin.create_dataset('start', data=np.concatenate(bin_start_list))
		# bin.create_dataset('end', data=np.concatenate(bin_end_list))
		# for cell in range(len(sc_cp_raw)):
		# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_raw[cell])
		
		# grp = output_f.create_group('compartment_zscore')
		# bin = grp.create_group('bin')
		# bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		#                    dtype=h5py.special_dtype(vlen=str))
		# bin.create_dataset('start', data=np.concatenate(bin_start_list))
		# bin.create_dataset('end', data=np.concatenate(bin_end_list))
		# for cell in range(len(sc_cp_all)):
		# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_zscore[cell])
		grp = output_f.create_group('compartment_bulk')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
		cell_type=cell_type_info['cell_type']
		i=0
		for j in list(set(cell_type)):
			grp.create_dataset(j, data=sc_cp_bulk[:,:,i])
			i+=1

			

	output_f.close()
	pool.shutdown(wait=True)

def test_compartment(matrix, return_PCA=False, model=None, expected = None):
	contact = matrix
	# np.fill_diagonal(contact, np.max(contact))
	# contact = KRnormalize(matrix)
	# contact[np.isnan(contact)] = 0.0
	contact = sqrt_norm(matrix)
	contact = oe(contact, expected)
	np.fill_diagonal(contact, 1)
	with warnings.catch_warnings():
		warnings.filterwarnings(
			"ignore", category=PearsonRConstantInputWarning
		)
		contact = pearson(contact)
	np.fill_diagonal(contact, 1)
	contact[np.isnan(contact)] = 0.0
	if model is not None:
		y = model.transform(contact)
	else:
		pca = PCA(n_components=3)
		y = pca.fit_transform(contact)
	if return_PCA:
		return y, pca
	else:
		return y


# args = parse_args()
# print (args)
# config = get_config('/home/python/higashi/cellcycle/1M/data/config_cellcycle.JSON')
config=get_config('/home/python/higashi/dataset2/cortex_250k_tmp/config_cellcycle.json')
res = config['resolution']
data_dir = config['data_dir']
temp_dir = config['temp_dir']
raw_dir = os.path.join(temp_dir, "raw")
if 'cytoband_path' in config:
	cytoband_path = config['cytoband_path']
else:
	cytoband_path = None
neighbor_num = config['neighbor_num']
embedding_name = config['embedding_name']

chrom_list = config['impute_list']
chrom_list = np.array(chrom_list)


# if args.calib:
# 	process_calib_file(args.calib_file)
start_call_compartment()

(782, 782)
(729, 729)
(641, 641)
(627, 627)
(608, 608)
(599, 599)
(582, 582)
(518, 518)
(499, 499)
(523, 523)
(489, 489)
(481, 481)
(482, 482)
(500, 500)
(417, 417)
(393, 393)
(380, 380)
(363, 363)
(246, 246)
chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19


NameError: name 'sc_cp_bulk' is not defined

In [64]:
chrom

'chr1'

In [61]:
origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)

In [66]:
origin_sparse[1074].shape

(782, 782)

In [68]:
cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
cell_type=cell_type_info['cell_type']
# sum_by_label=[]
a={}
b={}
for j in list(set(cell_type)):
    
    indices = [index for index, value in enumerate(cell_type) if value == j]
    a[j]=indices
    b[j] = np.zeros_like(np.array(origin_sparse[0]))
    for i in a[j]:
            proba = np.array(origin_sparse[i])
            b[j] +=proba

In [1]:
origin_sparse

NameError: name 'origin_sparse' is not defined

In [100]:
b['CX1']
array(<782x782 sparse matrix of type '<class 'numpy.float32'>'
	with 497258 stored elements in Compressed Sparse Row format>,
      dtype=object)
type(b['CX1'])
numpy.ndarray

array(<782x782 sparse matrix of type '<class 'numpy.float32'>'
	with 497258 stored elements in Compressed Sparse Row format>,
      dtype=object)

<782x782 sparse matrix of type '<class 'numpy.float32'>'
	with 583082 stored elements in Compressed Sparse Row format>

In [70]:
impute_f =  h5py.File(os.path.join(temp_dir, "%s_%s_nbr_0_impute.hdf5" % (chrom, embedding_name)),
						"r")
temp_compartment_list_all = [[] for i in range(len(use_rows_list))]
pre_bulk_list_all=[[] for i in range(len(use_rows_list))]

coordinates = impute_f['coordinates']
xs, ys = coordinates[:, 0], coordinates[:, 1]
cell_list = trange(len(list(impute_f.keys())) - 1)
temp = np.zeros((size, size))


temp *= 0.0
proba = np.array(impute_f["cell_%d" % 0])

  0%|          | 0/1075 [00:00<?, ?it/s]

In [72]:
proba.shape

(306153,)

In [113]:
type(b['CX1'].item())

scipy.sparse.csr.csr_matrix

In [109]:
b['CX1'][0]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [88]:
type(b['CX1'])

numpy.ndarray

In [103]:
dense_matrix = b['CX1'].toarray()

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [102]:
dense_matrices = [sparse_matrix.toarray() for sparse_matrix in b['CX1']]

TypeError: iteration over a 0-d array

In [117]:
temp =  np.array(b['CX1'].item().todense())
print(temp.shape)

(782, 782)


In [75]:
A=np.sum(origin_sparse, axis=0)

In [116]:
type(A)

scipy.sparse.csr.csr_matrix

In [83]:
temp = np.array(A.todense())
print(temp.shape)

(782, 782)


In [95]:
type(b['CX1'])

numpy.ndarray

In [97]:
b['CX1'].shape

()

In [85]:
############################################################
#
#不使用impute
#####使用原始的稀疏矩阵计算sc compartment和bulk compartment
##########################################################
#############################################################
#############################################################
##################################################################
##################################################################
from Higashi_backend.utils import *
from Higashi_analysis.Higashi_analysis import *
import h5py
from sklearn.preprocessing import MinMaxScaler, quantile_transform
import os

os.environ["OMP_NUM_THREADS"] = "10"
import pandas as pd
import argparse






def rankmatch(from_mtx, to_mtx):
	temp = np.sort(to_mtx.reshape((-1)))
	temp2 = from_mtx.reshape((-1))
	order = np.argsort(temp2)
	temp2[order] = temp
	return temp2.reshape((len(from_mtx), -1))


def create_mask(k=30, chrom="chr1", origin_sparse=None):
	final = np.array(np.sum(origin_sparse, axis=0).todense())
	size = origin_sparse[0].shape[-1]
	a = np.zeros((size, size))
	if k > 0:
		for i in range(min(k, len(a))):
			for j in range(len(a) - i):
				a[j, j + i] = 1
				a[j + i, j] = 1
		a = np.ones_like((a)) - a
	
	gap = np.sum(final, axis=-1, keepdims=False) == 0
	if cytoband_path is not None:
		gap_tab = pd.read_table(cytoband_path, sep="\t", header=None)
		gap_tab.columns = ['chrom', 'start', 'end', 'name', 'type']
		
		name = np.array(gap_tab['name'])
		# print (name)
		pqarm = np.array([str(s)[0] for s in name])
		gap_tab['pq_arm'] = pqarm
		gap_tab['length'] = gap_tab['end'] - gap_tab['start']
		summarize = gap_tab.groupby(['chrom', 'pq_arm']).sum().reset_index()
		# print (summarize)
		
		if np.sum(summarize['pq_arm'] == 'p') > 0:
			split_point = \
			np.ceil(np.array(summarize[(summarize['chrom'] == chrom) & (summarize['pq_arm'] == 'p')]['length']) / res)[0]
		else:
			split_point = -1
		
		gap_list = gap_tab[(gap_tab["chrom"] == chrom) & (gap_tab["type"] == "acen")]
		start = np.floor((np.array(gap_list['start'])) / res).astype('int')
		end = np.ceil((np.array(gap_list['end'])) / res).astype('int')
		
		for s, e in zip(start, end):
			a[s:e, :] = 1
			a[:, s:e] = 1
	else:
		split_point = -1
	a[gap, :] = 1
	a[:, gap] = 1
	
	return a, int(split_point)


def process_one_chrom(chrom):
	# Get the raw sparse mtx list
	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)
	size = origin_sparse[0].shape[0]
	# find centromere & gaps...
	mask, split_point = create_mask((int(1e5)), chrom, origin_sparse)

	bulk1 = np.array(np.sum(origin_sparse, axis=0).todense())
	print(bulk1.shape)
	mask = (np.ones_like(bulk1) - mask)
	bulk1 *= mask

	if "bulk_path" in config:
		import cooler
		c = cooler.Cooler("%s::resolutions/%d" % (config['bulk_path'], config['resolution']))
		bulk2 = np.array(c.matrix(sparse=False, balance=False).fetch(chrom)).astype('float')
		bulk2 *= mask
	else:
		bulk2 = None

	use_rows_all = []

	if split_point >= 20 * 1000000 / res:
		slice_start_list, slice_end_list = [0, split_point], [split_point, len(bulk1)]
	else:
		slice_start_list, slice_end_list = [0], [len(bulk1)]

	bulk_compartment_all = []
	real_bulk_compartment_all = []

	bulk_model_list = []
	bulk_reverse_list = []
	bulk_slice_list = []
	use_rows_list = []
	
	# temp_compartment_list_zscore = []
	# temp_compartment_list_quantile = []

	for slice_start, slice_end in zip(slice_start_list, slice_end_list):
		
		bulk1_slice = bulk1[slice_start:slice_end, :]
		bulk1_slice = bulk1_slice[:, slice_start:slice_end]
		use_rows = np.where(np.sum(bulk1_slice > 0, axis=-1) > 0.01 * len(bulk1_slice))[0]
		if len(use_rows) <= 1:
			print("no reliable bins in slice:", slice_start, slice_end)
			continue
		use_rows_all.append(np.arange(slice_start, slice_end)[use_rows])
		use_rows_list.append(use_rows)
		bulk1_slice = bulk1_slice[use_rows, :]
		bulk1_slice = bulk1_slice[:, use_rows]
		
		bulk_slice_list.append(bulk1_slice)
		bulk_expect = []
		for k in range(len(bulk1_slice)):
			diag = np.diag(bulk1_slice, k)
			bulk_expect.append(np.mean(diag))
		
		
		bulk_compartment, model = test_compartment(bulk1_slice, return_PCA=True)
		
		if bulk2 is not None:
			bulk2_slice = bulk2[slice_start:slice_end, :]
			bulk2_slice = bulk2_slice[:, slice_start:slice_end]
			bulk2_slice = bulk2_slice[use_rows, :]
			bulk2_slice = bulk2_slice[:, use_rows]
			real_bulk_compartment, model = test_compartment(bulk2_slice, return_PCA=True)
		else:
			real_bulk_compartment = None
			
		reverse_flag = False
		bulk_compartment_all.append(bulk_compartment)
		bulk_reverse_list.append(reverse_flag)
		bulk_model_list.append(model)
		real_bulk_compartment_all.append(real_bulk_compartment)
		


		
	temp_compartment_list_all = [[] for i in range(len(use_rows_list))]

	cell_list = trange(len(origin_sparse))
	# print(cell_list)
	temp = np.zeros((size, size))
	for i in cell_list:
		temp *= 0.0
		proba = np.array(origin_sparse[i].todense())
		temp+= proba
		temp = temp + temp.T
		temp *= mask
		
		for j in range(len(use_rows_list)):
			slice_start, slice_end = slice_start_list[j], slice_end_list[j]
			temp_slice = temp[slice_start:slice_end, :]
			temp_slice = temp_slice[:, slice_start:slice_end]
			temp_select = temp_slice[use_rows_list[j], :]
			temp_select = temp_select[:, use_rows_list[j]]
			# temp_select = rankmatch(temp_select, bulk_slice_list[j])
			temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
			if bulk_reverse_list[j]:
				temp_compartment = -1 * temp_compartment
			# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
			temp_compartment_list_all[j].append(temp_compartment)
			
	for j in range(len(use_rows_list)):
		temp_compartment_list_all[j] = np.stack(temp_compartment_list_all[j], axis=0)
		

	real_bulk_compartment = np.concatenate(real_bulk_compartment_all, axis=0) if bulk2 is not None else None
	bulk_compartment = np.concatenate(bulk_compartment_all, axis=0)
	temp_compartment_list = np.concatenate(temp_compartment_list_all, axis=-1)
	use_rows = np.concatenate(use_rows_all, axis=0)
	print (chrom, "finished")
	# print("temp_compartment_list shape:",temp_compartment_list.shape)

	##########从原始的接触图计算
	origin_bulk_list_all=[[] for i in range(len(use_rows_list))]
	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	# sum_by_label=[]
	a={}
	b={}
	for j in list(set(cell_type)):
		
		indices = [index for index, value in enumerate(cell_type) if value == j]
		a[j]=indices
		b[j] = np.zeros_like(np.array(origin_sparse[0]))
		for i in a[j]:
				proba = np.array(origin_sparse[i])    
				b[j] +=proba
		temp = np.array(b[j].item().todense())
		# print(temp.shape)
		temp *= mask
		for j in range(len(use_rows_list)):
			slice_start, slice_end = slice_start_list[j], slice_end_list[j]
			temp_slice = temp[slice_start:slice_end, :]
			temp_slice = temp_slice[:, slice_start:slice_end]
			temp_select = temp_slice[use_rows_list[j], :]
			temp_select = temp_select[:, use_rows_list[j]]
			# temp_select = rankmatch(temp_select, bulk_slice_list[j])
			temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
			if bulk_reverse_list[j]:
				temp_compartment = -1 * temp_compartment
			# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
			origin_bulk_list_all[j].append(temp_compartment)
        
	for j in range(len(use_rows_list)):
		origin_bulk_list_all[j] = np.stack(origin_bulk_list_all[j], axis=0)
	origin_bulk_list = np.stack(origin_bulk_list_all[j], axis=-1)

	return real_bulk_compartment, bulk_compartment, chrom, use_rows, size,origin_bulk_list,temp_compartment_list

	
			

def process_calib_file(file_path):
	tab = pd.read_table(file_path , sep="\t", header=None)
	tab.columns = ['chrom', 'bin', 'value']
	# print(tab)
	chrom_start_end = np.load(os.path.join(temp_dir, "chrom_start_end.npy"))
	tab['chrom'] = np.array(tab['chrom']).astype('str')
	calib_result = {}
	for i, chrom in enumerate(chrom_list):
		temp = tab[tab['chrom'] == chrom]
		size = chrom_start_end[i, 1] - chrom_start_end[i, 0]
		vec = np.zeros(size)
		indice = (np.array(temp['bin'] / res)).astype('int')
		v = np.array(temp['value'])
		v[v == -1] = np.nan
		vec[indice] = v
		calib_result[chrom] = vec
	np.save(os.path.join(temp_dir, "calib.npy"), calib_result, allow_pickle=True)


def start_call_compartment(output):
	p_list = []
	pool = ProcessPoolExecutor(max_workers=10)
	
	if ".hdf5" not in output:
		output += ".hdf5"
	with h5py.File(os.path.join(temp_dir, output), "w") as output_f:
		result = {}
		for chrom in chrom_list:
			# real_bulk_compartment, bulk_compartment, temp_compartment_list, temp_compartment_zscore, temp_compartment_quantile, chrom, use_rows, size = process_one_chrom(chrom)
			real_bulk_compartment, bulk_compartment, chrom, use_rows, size,origin_bulk_list,temp_compartment_list = process_one_chrom(chrom)
		# 	p_list.append(pool.submit(process_one_chrom, chrom))
		# 
		# 
		# for p in as_completed(p_list):
		# 	real_bulk_compartment, bulk_compartment, temp_compartment_list, temp_compartment_zscore, temp_compartment_quantile, chrom, use_rows, size = p.result()
			result[chrom] = [real_bulk_compartment, bulk_compartment, use_rows, size,origin_bulk_list,temp_compartment_list]
			
		bin_chrom_list = []
		bin_start_list = []
		bin_end_list = []
		bulk_cp_all = []
		real_bulk_cp_all = []
		sc_cp_all = []
		sc_cp_raw = []
		sc_cp_zscore = []
		sc_cp_bulk = []
		grp = output_f.create_group('compartment')
		bin = grp.create_group('bin')
		
		for chrom in chrom_list:
			real_bulk_compartment, bulk_compartment, use_rows, size,origin_bulk_list ,temp_compartment_list= result[chrom]
			# print (use_rows)
			# print(chrom)
			length = size
			bin_chrom_list += [chrom] * len(use_rows)
			bin_start_list.append((np.arange(length) * res).astype('int')[use_rows])
			bin_end_list.append(((np.arange(length) + 1) * res).astype('int')[use_rows])
			bulk_cp_all.append(bulk_compartment)
			real_bulk_cp_all.append(real_bulk_compartment)
			# sc_cp_all.append(temp_compartment_quantile)
			
			# print("temp_compartment_list shap to append:",temp_compartment_list.shape)
			sc_cp_raw.append(temp_compartment_list)
			# print("sc_cp_raw shape",(np.array(sc_cp_raw)).shape)
			sc_cp_bulk.append(origin_bulk_list)
			
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		# print("bulk_cp_all:",np.array(bulk_cp_all).shape)
		bulk_cp_all = np.concatenate(bulk_cp_all, axis=0)
		# print("bulk_cp_all shape:",np.array(bulk_cp_all).shape)
		grp.create_dataset("bulk", data=bulk_cp_all)
		
		if real_bulk_compartment is not None:
			real_bulk_cp_all = np.concatenate(real_bulk_cp_all, axis=0)
			grp.create_dataset("real_bulk", data=real_bulk_cp_all)
		
		# sc_cp_all = np.concatenate(sc_cp_all, axis=-1)
		sc_cp_raw = np.concatenate(sc_cp_raw, axis=-2)
		# print('sc_cp_raw:',(np.array(sc_cp_raw)).shape)
		# sc_cp_zscore = np.concatenate(sc_cp_zscore, axis=-1)
		sc_cp_bulk=np.concatenate(sc_cp_bulk, axis=0)
		# print('sc_cp_bulk:',(np.array(sc_cp_bulk)).shape)
		
		# for cell in range(len(sc_cp_all)):
		# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_all[cell])
		
		grp = output_f.create_group('compartment_raw')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		for cell in range(len(sc_cp_raw)):
			grp.create_dataset("cell_%d" % cell, data=sc_cp_raw[cell])
		
		# grp = output_f.create_group('compartment_zscore')
		# bin = grp.create_group('bin')
		# bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		#                    dtype=h5py.special_dtype(vlen=str))
		# bin.create_dataset('start', data=np.concatenate(bin_start_list))
		# bin.create_dataset('end', data=np.concatenate(bin_end_list))
		# for cell in range(len(sc_cp_all)):
		# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_zscore[cell])
		grp = output_f.create_group('compartment_bulk')
		bin = grp.create_group('bin')
		bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		                   dtype=h5py.special_dtype(vlen=str))
		bin.create_dataset('start', data=np.concatenate(bin_start_list))
		bin.create_dataset('end', data=np.concatenate(bin_end_list))
		cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
		cell_type=cell_type_info['cell_type']
		i=0
		for j in list(set(cell_type)):
			grp.create_dataset(j, data=sc_cp_bulk[:,:,i])
			i+=1  

			

	output_f.close()
	pool.shutdown(wait=True)

def test_compartment(matrix, return_PCA=False, model=None, expected = None):
	contact = matrix
	# np.fill_diagonal(contact, np.max(contact))
	# contact = KRnormalize(matrix)
	# contact[np.isnan(contact)] = 0.0
	contact = sqrt_norm(matrix)
	contact = oe(contact, expected)
	np.fill_diagonal(contact, 1)
	with warnings.catch_warnings():
		warnings.filterwarnings(
			"ignore", category=PearsonRConstantInputWarning
		)
		contact = pearson(contact)
	np.fill_diagonal(contact, 1)
	contact[np.isnan(contact)] = 0.0
	if model is not None:
		y = model.transform(contact)
	else:
		pca = PCA(n_components=3)
		y = pca.fit_transform(contact)
	if return_PCA:
		return y, pca
	else:
		return y


# args = parse_args()
# print (args)
# config = get_config('/home/python/higashi/cellcycle/1M/data/config_cellcycle.JSON')
output = 'sc_and_bulk_cortex250k_new'
config=get_config('/home/python/higashi/dataset2/cortex_250k_tmp/config_cellcycle.json')
res = config['resolution']
data_dir = config['data_dir']
temp_dir = config['temp_dir']
raw_dir = os.path.join(temp_dir, "raw")
if 'cytoband_path' in config:
	cytoband_path = config['cytoband_path']
else:
	cytoband_path = None
neighbor_num = config['neighbor_num']
embedding_name = config['embedding_name']

chrom_list = config['impute_list']
chrom_list = np.array(chrom_list)


# if args.calib:
# 	process_calib_file(args.calib_file)
start_call_compartment(output)

(782, 782)


  3%|▎         | 30/1075 [00:06<03:29,  4.98it/s]


KeyboardInterrupt: 

In [81]:
config=get_config('/home/python/higashi/dataset2/cortex_250k_tmp/config_cellcycle.json')
res = config['resolution']
data_dir = config['data_dir']
temp_dir = config['temp_dir']
raw_dir = os.path.join(temp_dir, "raw")
if 'cytoband_path' in config:
	cytoband_path = config['cytoband_path']
else:
	cytoband_path = None
neighbor_num = config['neighbor_num']
embedding_name = config['embedding_name']

chrom_list = config['impute_list']
chrom_list = np.array(chrom_list)

In [82]:
cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
cell_type=cell_type_info['cell_type']
origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)




In [86]:
origin_sparse[0].data.shape

(14887,)

In [2]:
X=h5py.File('/home/python/higashi/dataset2/cortex_250k_tmp/data/compartment_sc_and_bulk_cortex250k.hdf5', 'r')

In [4]:
group='compartment_raw'
X[group].keys()

<KeysViewHDF5 ['bin', 'cell_0', 'cell_1', 'cell_10', 'cell_100', 'cell_1000', 'cell_1001', 'cell_1002', 'cell_1003', 'cell_1004', 'cell_1005', 'cell_1006', 'cell_1007', 'cell_1008', 'cell_1009', 'cell_101', 'cell_1010', 'cell_1011', 'cell_1012', 'cell_1013', 'cell_1014', 'cell_1015', 'cell_1016', 'cell_1017', 'cell_1018', 'cell_1019', 'cell_102', 'cell_1020', 'cell_1021', 'cell_1022', 'cell_1023', 'cell_1024', 'cell_1025', 'cell_1026', 'cell_1027', 'cell_1028', 'cell_1029', 'cell_103', 'cell_1030', 'cell_1031', 'cell_1032', 'cell_1033', 'cell_1034', 'cell_1035', 'cell_1036', 'cell_1037', 'cell_1038', 'cell_1039', 'cell_104', 'cell_1040', 'cell_1041', 'cell_1042', 'cell_1043', 'cell_1044', 'cell_1045', 'cell_1046', 'cell_1047', 'cell_1048', 'cell_1049', 'cell_105', 'cell_1050', 'cell_1051', 'cell_1052', 'cell_1053', 'cell_1054', 'cell_1055', 'cell_1056', 'cell_1057', 'cell_1058', 'cell_1059', 'cell_106', 'cell_1060', 'cell_1061', 'cell_1062', 'cell_1063', 'cell_1064', 'cell_1065', 'cell

In [22]:
X[group]['cell_0'][:,0].shape

(9619,)

In [77]:
cell_list = trange(len(origin_sparse))

  0%|          | 0/1172 [00:00<?, ?it/s]

In [79]:
for i in cell_list:
    print(i)

100%|██████████| 1172/1172 [00:26<00:00, 43.57it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# cell_list = trange(len(list(impute_f.keys())) - 1)
	# temp = np.zeros((size, size))

In [5]:
# from Higashi_backend.utils import *
# from Higashi_analysis.Higashi_analysis import *
# import h5py
# from sklearn.preprocessing import MinMaxScaler, quantile_transform
# import os

# os.environ["OMP_NUM_THREADS"] = "10"
# import pandas as pd
# import argparse

# config=get_config('/home/python/higashi/dataset2/cortex_250k_tmp/config_cellcycle.json')
# res = config['resolution']
# data_dir = config['data_dir']
# temp_dir = config['temp_dir']
# raw_dir = os.path.join(temp_dir, "raw")
# if 'cytoband_path' in config:
# 	cytoband_path = config['cytoband_path']
# else:
# 	cytoband_path = None
# neighbor_num = config['neighbor_num']
# embedding_name = config['embedding_name']

# chrom_list = config['impute_list']
# chrom_list = np.array(chrom_list)


# chrom ='chr1'

# cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
# cell_type=cell_type_info['cell_type']
	
# origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)

In [7]:
len(origin_sparse)

1075

In [20]:
origin_sparse[0]

<782x782 sparse matrix of type '<class 'numpy.float32'>'
	with 14887 stored elements in Compressed Sparse Row format>

In [15]:
cell_list = trange(len(origin_sparse))



  0%|          | 0/1074 [00:07<?, ?it/s]


In [11]:
cell_list

In [17]:
impute_f =  h5py.File(os.path.join(temp_dir, "%s_%s_nbr_0_impute.hdf5" % (chrom, embedding_name)),
						"r")

In [18]:
len(list(impute_f.keys()))

1076

In [55]:
############################################################
#
#不使用impute
#####提取prebulk稀疏矩阵，不计算pc值
##########################################################
#############################################################
#############################################################
##################################################################
##################################################################
from Higashi_backend.utils import *
from Higashi_analysis.Higashi_analysis import *
import h5py
from sklearn.preprocessing import MinMaxScaler, quantile_transform
import os

os.environ["OMP_NUM_THREADS"] = "10"
import pandas as pd
import argparse

import scipy.sparse
# adj_sp=scipy.sparse.csr_matrix(adj)






def rankmatch(from_mtx, to_mtx):
	temp = np.sort(to_mtx.reshape((-1)))
	temp2 = from_mtx.reshape((-1))
	order = np.argsort(temp2)
	temp2[order] = temp
	return temp2.reshape((len(from_mtx), -1))


def create_mask(k=30, chrom="chr1", origin_sparse=None):
	final = np.array(np.sum(origin_sparse, axis=0).todense())
	size = origin_sparse[0].shape[-1]
	a = np.zeros((size, size))
	if k > 0:
		for i in range(min(k, len(a))):
			for j in range(len(a) - i):
				a[j, j + i] = 1
				a[j + i, j] = 1
		a = np.ones_like((a)) - a
	
	gap = np.sum(final, axis=-1, keepdims=False) == 0
	if cytoband_path is not None:
		gap_tab = pd.read_table(cytoband_path, sep="\t", header=None)
		gap_tab.columns = ['chrom', 'start', 'end', 'name', 'type']
		
		name = np.array(gap_tab['name'])
		# print (name)
		pqarm = np.array([str(s)[0] for s in name])
		gap_tab['pq_arm'] = pqarm
		gap_tab['length'] = gap_tab['end'] - gap_tab['start']
		summarize = gap_tab.groupby(['chrom', 'pq_arm']).sum().reset_index()
		# print (summarize)
		
		if np.sum(summarize['pq_arm'] == 'p') > 0:
			split_point = \
			np.ceil(np.array(summarize[(summarize['chrom'] == chrom) & (summarize['pq_arm'] == 'p')]['length']) / res)[0]
		else:
			split_point = -1
		
		gap_list = gap_tab[(gap_tab["chrom"] == chrom) & (gap_tab["type"] == "acen")]
		start = np.floor((np.array(gap_list['start'])) / res).astype('int')
		end = np.ceil((np.array(gap_list['end'])) / res).astype('int')
		
		for s, e in zip(start, end):
			a[s:e, :] = 1
			a[:, s:e] = 1
	else:
		split_point = -1
	a[gap, :] = 1
	a[:, gap] = 1
	
	return a, int(split_point)


def process_one_chrom(chrom):
	# Get the raw sparse mtx list
	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)
	size = origin_sparse[0].shape[0]
	# find centromere & gaps...
	mask, split_point = create_mask((int(1e5)), chrom, origin_sparse)

	bulk1 = np.array(np.sum(origin_sparse, axis=0).todense())
	print(bulk1.shape)
	mask = (np.ones_like(bulk1) - mask)
	bulk1 *= mask

	if "bulk_path" in config:
		import cooler
		c = cooler.Cooler("%s::resolutions/%d" % (config['bulk_path'], config['resolution']))
		bulk2 = np.array(c.matrix(sparse=False, balance=False).fetch(chrom)).astype('float')
		bulk2 *= mask
	else:
		bulk2 = None

	use_rows_all = []

	if split_point >= 20 * 1000000 / res:
		slice_start_list, slice_end_list = [0, split_point], [split_point, len(bulk1)]
	else:
		slice_start_list, slice_end_list = [0], [len(bulk1)]

	bulk_compartment_all = []
	real_bulk_compartment_all = []

	bulk_model_list = []
	bulk_reverse_list = []
	bulk_slice_list = []
	use_rows_list = []
	
	# temp_compartment_list_zscore = []
	# temp_compartment_list_quantile = []

	for slice_start, slice_end in zip(slice_start_list, slice_end_list):
		
		bulk1_slice = bulk1[slice_start:slice_end, :]
		bulk1_slice = bulk1_slice[:, slice_start:slice_end]
		use_rows = np.where(np.sum(bulk1_slice > 0, axis=-1) > 0.01 * len(bulk1_slice))[0]
		if len(use_rows) <= 1:
			print("no reliable bins in slice:", slice_start, slice_end)
			continue
		use_rows_all.append(np.arange(slice_start, slice_end)[use_rows])
		use_rows_list.append(use_rows)
		bulk1_slice = bulk1_slice[use_rows, :]
		bulk1_slice = bulk1_slice[:, use_rows]
		
		bulk_slice_list.append(bulk1_slice)
		bulk_expect = []
		for k in range(len(bulk1_slice)):
			diag = np.diag(bulk1_slice, k)
			bulk_expect.append(np.mean(diag))
		
		
		bulk_compartment, model = test_compartment(bulk1_slice, return_PCA=True)
		
		if bulk2 is not None:
			bulk2_slice = bulk2[slice_start:slice_end, :]
			bulk2_slice = bulk2_slice[:, slice_start:slice_end]
			bulk2_slice = bulk2_slice[use_rows, :]
			bulk2_slice = bulk2_slice[:, use_rows]
			real_bulk_compartment, model = test_compartment(bulk2_slice, return_PCA=True)
		else:
			real_bulk_compartment = None
			
		reverse_flag = False
		bulk_compartment_all.append(bulk_compartment)
		bulk_reverse_list.append(reverse_flag)
		bulk_model_list.append(model)
		real_bulk_compartment_all.append(real_bulk_compartment)
		

	# print (chrom, "finished")
	# print("temp_compartment_list shape:",temp_compartment_list.shape)

	##########从原始的接触图计算
	origin_bulk_list_all=[[] for i in range(len(use_rows_list))]
	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	cell_type=cell_type_info['cell_type']
	# sum_by_label=[]
	a={}
	b={}
	for j in list(set(cell_type)):
		
		indices = [index for index, value in enumerate(cell_type) if value == j]
		a[j]=indices
		b[j] = np.zeros_like(np.array(origin_sparse[0]))
		for i in a[j]:
				proba = np.array(origin_sparse[i])
				b[j] +=proba
		temp = np.array(b[j].item().todense())
		temp *= mask
		for j in range(len(use_rows_list)):
			slice_start, slice_end = slice_start_list[j], slice_end_list[j]
			temp_slice = temp[slice_start:slice_end, :]
			temp_slice = temp_slice[:, slice_start:slice_end]
			temp_select = temp_slice[use_rows_list[j], :]
			temp_compartment = temp_select[:, use_rows_list[j]]
			# temp_select = rankmatch(temp_select, bulk_slice_list[j])
			# temp_compartment = test_compartment(temp_select, False, bulk_model_list[j], None)
			if bulk_reverse_list[j]:
				temp_compartment = -1 * temp_compartment
			# temp_compartment_list_all[j].append(temp_compartment.reshape((-1)))
			origin_bulk_list_all[j].append(temp_compartment)
        
	for j in range(len(use_rows_list)):
		origin_bulk_list_all[j] = np.stack(origin_bulk_list_all[j], axis=0)

	origin_bulk_list = np.stack(origin_bulk_list_all[j], axis=-1)
	print('origin_bulk_list shape:',origin_bulk_list.shape)
	# adj_sp=scipy.sparse.csr_matrix(origin_bulk_list )
	# print('adj_sp size',adj_sp.size)
	# 初始化一个稀疏矩阵
	sparse_matrix_list = []
	i=0
	for j in list(set(cell_type)):
		current_array=origin_bulk_list[:,:,i]
		current_sparse_matrix = scipy.sparse.csr_matrix(np.reshape(current_array, (current_array.shape[0], -1)))
		# 将当前稀疏矩阵添加到稀疏矩阵列表中

		sparse_matrix_list.append([current_sparse_matrix.data])
		# # 将当前稀疏矩阵添加到整体稀疏矩阵中
		# if sparse_matrix is None:
		# 	sparse_matrix = current_sparse_matrix
		# else:
		# 	sparse_matrix = scipy.sparse.vstack([sparse_matrix, current_sparse_matrix], format='csr')
		# print('current_sparse_matrix.data',current_sparse_matrix.data)
		i += 1
	# print('sparse_matrix ',np.array(sparse_matrix_list, dtype=object).shape )
	print(chrom, "finished")


	return real_bulk_compartment, bulk_compartment, chrom, use_rows,size,origin_bulk_list,sparse_matrix_list

	
			

def process_calib_file(file_path):
	tab = pd.read_table(file_path , sep="\t", header=None)
	tab.columns = ['chrom', 'bin', 'value']
	# print(tab)
	chrom_start_end = np.load(os.path.join(temp_dir, "chrom_start_end.npy"))
	tab['chrom'] = np.array(tab['chrom']).astype('str')
	calib_result = {}
	for i, chrom in enumerate(chrom_list):
		temp = tab[tab['chrom'] == chrom]
		size = chrom_start_end[i, 1] - chrom_start_end[i, 0]
		vec = np.zeros(size)
		indice = (np.array(temp['bin'] / res)).astype('int')
		v = np.array(temp['value'])
		v[v == -1] = np.nan
		vec[indice] = v
		calib_result[chrom] = vec
	np.save(os.path.join(temp_dir, "calib.npy"), calib_result, allow_pickle=True)


def start_call_compartment():
	p_list = []
	pool = ProcessPoolExecutor(max_workers=10)
	output='test'
	if ".hdf5" not in output:
		output += ".hdf5"
	with h5py.File(os.path.join(temp_dir, output), "w") as output_f:
		result = {}
		for chrom in chrom_list:
			# real_bulk_compartment, bulk_compartment, temp_compartment_list, temp_compartment_zscore, temp_compartment_quantile, chrom, use_rows, size = process_one_chrom(chrom)
			real_bulk_compartment, bulk_compartment, chrom, use_rows, size,origin_bulk_list,sparse_matrix_list= process_one_chrom(chrom)
		# 	p_list.append(pool.submit(process_one_chrom, chrom))
		# 
		# 
		# for p in as_completed(p_list):
		# 	real_bulk_compartment, bulk_compartment, temp_compartment_list, temp_compartment_zscore, temp_compartment_quantile, chrom, use_rows, size = p.result()
			result[chrom] = [real_bulk_compartment, bulk_compartment, use_rows, size,origin_bulk_list,sparse_matrix_list]
			
		bin_chrom_list = []
		bin_start_list = []
		bin_end_list = []
		bulk_cp_all = []
		real_bulk_cp_all = []
		bin_value = []
		sc_cp_all = []
		sc_cp_raw = []
		sc_cp_zscore = []
		sc_cp_bulk = []
		
		for chrom in chrom_list:
			real_bulk_compartment, bulk_compartment, use_rows, size,origin_bulk_list,sparse_matrix_list= result[chrom]
			# print (use_rows)
			# print(chrom)
			length = size
			bin_chrom_list += [chrom] * len(use_rows)
			bin_start_list.append((np.arange(length) * res).astype('int')[use_rows])
			bin_end_list.append(((np.arange(length) + 1) * res).astype('int')[use_rows])
			bulk_cp_all.append(bulk_compartment)
			real_bulk_cp_all.append(real_bulk_compartment)
			bin_value.append(sparse_matrix_list)

		# result_bin = [element for sublist in zip(bin_value) for element in sublist]
		# print('result_bin:',result_bin)
		# result_bin = []
		# for sublist in zip(*bin_value):
		# 	group = []
		# 	for element in sublist:
		# 		group.extend(element)
		# 	result_bin.append(group)
		# result_bin = np.concatenate(bin_value, axis=0)
		# # 使用列表解析获取每个数组的形状
		# shapes = [arr[0].shape for sublist in result_bin for arr in sublist]

		# # 输出结果
		# for i, shape in enumerate(shapes):
		# 	print(f"数组 {i + 1} 的形状：{shape}")
		# 使用列表解析将每个数组放入一个列表中
		result_bin = [np.array(value) for value in bin_value]

		# 输出结果
		for i, array in enumerate(result_bin):
			print(f"数组 {i + 1} 的形状：{array.shape}")
		# sc_cp_bulk=np.concatenate(bin_value, axis=0)
		# print(np.array(sc_cp_bulk).shape)
			# print('bin_value shape',np.array(bin_value).shape)
			# # sc_cp_all.append(temp_compartment_quantile)
			
			# # print("temp_compartment_list shap to append:",temp_compartment_list.shape)
			# sc_cp_raw.append(temp_compartment_list)
			# # print("sc_cp_raw shape",(np.array(sc_cp_raw)).shape)
			# sc_cp_bulk.append(origin_bulk_list)
			
		# bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
		#                    dtype=h5py.special_dtype(vlen=str))
		# bin.create_dataset('start', data=np.concatenate(bin_start_list))
		# bin.create_dataset('end', data=np.concatenate(bin_end_list))
		# # print("bulk_cp_all:",np.array(bulk_cp_all).shape)
		# bulk_cp_all = np.concatenate(bulk_cp_all, axis=0)
		# # print("bulk_cp_all shape:",np.array(bulk_cp_all).shape)
		# grp.create_dataset("bulk", data=bulk_cp_all)
		
		# if real_bulk_compartment is not None:
		# 	real_bulk_cp_all = np.concatenate(real_bulk_cp_all, axis=0)
		# 	grp.create_dataset("real_bulk", data=real_bulk_cp_all)
		
		# # sc_cp_all = np.concatenate(sc_cp_all, axis=-1)
		# sc_cp_raw = np.concatenate(sc_cp_raw, axis=-2)
		# # print('sc_cp_raw:',(np.array(sc_cp_raw)).shape)
		# # sc_cp_zscore = np.concatenate(sc_cp_zscore, axis=-1)
		# sc_cp_bulk=np.concatenate(sc_cp_bulk, axis=0)
		# print('sc_cp_bulk:',(np.array(sc_cp_bulk)).shape)
		
		# for cell in range(len(sc_cp_all)):
		# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_all[cell])
		
	# 	grp = output_f.create_group('compartment_raw')
	# 	bin = grp.create_group('bin')
	# 	bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
	# 	                   dtype=h5py.special_dtype(vlen=str))
	# 	bin.create_dataset('start', data=np.concatenate(bin_start_list))
	# 	bin.create_dataset('end', data=np.concatenate(bin_end_list))
	# 	for cell in range(len(sc_cp_raw)):
	# 		grp.create_dataset("cell_%d" % cell, data=sc_cp_raw[cell])
		
	# 	# grp = output_f.create_group('compartment_zscore')
	# 	# bin = grp.create_group('bin')
	# 	# bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
	# 	#                    dtype=h5py.special_dtype(vlen=str))
	# 	# bin.create_dataset('start', data=np.concatenate(bin_start_list))
	# 	# bin.create_dataset('end', data=np.concatenate(bin_end_list))
	# 	# for cell in range(len(sc_cp_all)):
	# 	# 	grp.create_dataset("cell_%d" % cell, data=sc_cp_zscore[cell])

	# 	grp = output_f.create_group('compartment_bulk')
	# 	bin = grp.create_group('bin')
	# 	bin.create_dataset('chrom', data=[l.encode('utf8') for l in bin_chrom_list],
	# 	                   dtype=h5py.special_dtype(vlen=str))
	# 	bin.create_dataset('start', data=np.concatenate(bin_start_list))
	# 	bin.create_dataset('end', data=np.concatenate(bin_end_list))
	# 	cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
	# 	cell_type=cell_type_info['cell_type']
	# 	i=0
	# 	for j in list(set(cell_type)):
	# 		grp.create_dataset(j, data=sc_cp_bulk[:,:,i])
	# 		i+=1		
	# # output_f.close()
	# 	pool.shutdown(wait=True)

def test_compartment(matrix, return_PCA=False, model=None, expected = None):
	contact = matrix
	# np.fill_diagonal(contact, np.max(contact))
	# contact = KRnormalize(matrix)
	# contact[np.isnan(contact)] = 0.0
	contact = sqrt_norm(matrix)
	contact = oe(contact, expected)
	np.fill_diagonal(contact, 1)
	with warnings.catch_warnings():
		warnings.filterwarnings(
			"ignore", category=PearsonRConstantInputWarning
		)
		contact = pearson(contact)
	np.fill_diagonal(contact, 1)
	contact[np.isnan(contact)] = 0.0
	if model is not None:
		y = model.transform(contact)
	else:
		pca = PCA(n_components=3)
		y = pca.fit_transform(contact)
	if return_PCA:
		return y, pca
	else:
		return y


# args = parse_args()
# print (args)
config = get_config('/home/python/higashi/cellcycle/1M/data/config_cellcycle.JSON')
# output = 'sc_and_bulk_cortex250k'
# config=get_config('/home/python/higashi/dataset2/cortex_250k_tmp/config_cellcycle.json')
res = config['resolution']
data_dir = config['data_dir']
temp_dir = config['temp_dir']
raw_dir = os.path.join(temp_dir, "raw")
if 'cytoband_path' in config:
	cytoband_path = config['cytoband_path']
else:
	cytoband_path = None
neighbor_num = config['neighbor_num']
embedding_name = config['embedding_name']

chrom_list = config['impute_list']
chrom_list = np.array(chrom_list)


# if args.calib:
# 	process_calib_file(args.calib_file)
start_call_compartment()

(198, 198)
origin_bulk_list shape: (194, 194, 4)
chr1 finished
(182, 182)
origin_bulk_list shape: (179, 179, 4)
chr2 finished
数组 1 的形状：(4, 1)
数组 2 的形状：(4, 1)


/tmp/ipykernel_30894/95344279.py:300: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result_bin = [np.array(value) for value in bin_value]


In [40]:
# 假设有两个列表，分别包含四组长度为198和四组长度为164的向量
list1 = [[1]*198, [2]*198, [3]*198, [4]*198]
list2 = [[5]*164, [6]*164, [7]*164, [8]*164]

# 使用 zip 函数将对应位置的向量组合
combined_lists = zip(list1, list2)

# 使用列表解析按组拼接向量
result = [vector1 + vector2 for vector1, vector2 in combined_lists]

# 输出结果
for i, combined_vector in enumerate(result):
    print(f"组合向量 {i + 1} 的长度：{len(combined_vector)}")


组合向量 1 的长度：362
组合向量 2 的长度：362
组合向量 3 的长度：362
组合向量 4 的长度：362


In [56]:
chrom='chr1'
cell_type_info=pickle.load(open(os.path.join(data_dir, "label_info.pickle"), "rb"))
cell_type=cell_type_info['cell_type']
origin_sparse = np.load(os.path.join(raw_dir, "%s_sparse_adj.npy" % chrom), allow_pickle=True)


In [76]:
j='1CDX1'
indices = [index for index, value in enumerate(cell_type) if value == j]

In [74]:
indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [58]:
size = origin_sparse[0].shape
size

(198, 198)

In [72]:
origin_sparse[19].data.shape


(1431,)